---
###Funkcje
---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [ ]:
def convert_hour_to_minutes(df, column_name):
  df[column_name] = df[column_name].apply(
      lambda x: (int(x) // 100) * 60 + (int(x) % 100))

In [ ]:
def encode_cyclic_feature(df, column_name, max_value):
  df[f'{column_name}_sin'] = df[column_name].apply(
    lambda x: np.sin(2 * np.pi * x / max_value))
  df[f'{column_name}_cos'] = df[column_name].apply(
    lambda x: np.cos(2 * np.pi * x / max_value))
  df.drop(column_name, axis=1, inplace=True)

In [ ]:
def categorize_value(value):
  if value < 10:
    return 0
  else:
    return 1

In [1]:
def grid_search(model, parameters, X_train, y_train):
    grid_search = GridSearchCV(model,
                               param_grid=parameters,
                               cv=10,
                               n_jobs=-1)
    grid_search.fit(X_train, y_train)

    return grid_search

In [ ]:
def history_to_df(history):
  df_history = pd.DataFrame(history.history)
  df_history['epoch'] = df_history.index + 1

In [ ]:
def plt_history(df):

  for i in range(len(df.columns) // 2):
    metric = df.columns[i]

    if metric == 'loss':
      best_epoch = df[f'val_{metric}'].idxmin() + 1
    else:
      best_epoch = df[f'val_{metric}'].idxmax() + 1

    print(f'best_epoch dla {metric}:', best_epoch)

    plt.figure(figsize=(15,3))
    plt.plot(df['epoch'], df[metric], label=f'{metric.capitalize()} treningowe',
             marker='o')
    plt.plot(df['epoch'], df[f'val_{metric}'],
             label=f'{metric.capitalize()} treningowe',
             marker='o')
    plt.axvline(best_epoch, color='red', linestyle='--',
                label=f'Best epoch dla {metric}')
    plt.title(f'{metric.capitalize()} na zbiorze treningowym i walidacyjnym')
    plt.xlabel('Epoch')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.grid()
    plt.show()